In [1]:
!sudo apt-get install build-essential swig
!curl https://raw.githubusercontent.com/automl/auto-sklearn/master/requirements.txt | xargs -n 1 -L 1 pip install
!pip install auto-sklearn

Reading package lists... Done
Building dependency tree       
Reading state information... Done
build-essential is already the newest version (12.4ubuntu1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
Suggested packages:
  swig-doc swig-examples swig3.0-examples swig3.0-doc
The following NEW packages will be installed:
  swig swig3.0
0 upgraded, 2 newly installed, 0 to remove and 49 not upgraded.
Need to get 1,100 kB of archives.
After this operation, 5,822 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 swig3.0 amd64 3.0.12-1 [1,094 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 swig amd64 3.0.12-1 [6,460 B]
Fetched 1,100 kB in 4s (280 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontE

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np



In [2]:
churn_df = pd.read_csv(r'/content/drive/MyDrive/3 AutoML/WA_Fn-UseC_-Telco-Customer-Churn.csv')

In [3]:
churn_df = churn_df.replace(r'^\s*$', np.nan, regex=True)
churn_df.iloc[:, 19] = pd.to_numeric(churn_df.iloc[:, 19], errors='coerce')

from sklearn.impute import SimpleImputer
imp_median = SimpleImputer(missing_values=np.nan, strategy='median')
churn_df.iloc[:, 19] = imp_median.fit_transform(churn_df.iloc[:, 19].values.reshape(-1, 1) )

In [4]:
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OrdinalEncoder

categorical_columns = ['gender', 'Partner', 'Dependents','PhoneService','MultipleLines','InternetService','OnlineSecurity','OnlineBackup','DeviceProtection','TechSupport','StreamingTV','StreamingMovies','Contract','PaperlessBilling','PaymentMethod','Churn']
column_trans = make_column_transformer((OrdinalEncoder(), categorical_columns))

churn_transformed=column_trans.fit_transform(churn_df)


In [5]:
churn_df_trans = churn_df.copy()
churn_df_trans = pd.DataFrame(churn_transformed, columns=categorical_columns).astype(int)

In [6]:
churn_df.update(churn_df_trans)

In [49]:
l = ['gender', 'Partner', 'Dependents','PhoneService','MultipleLines','InternetService','OnlineSecurity','OnlineBackup','DeviceProtection','TechSupport','StreamingTV','StreamingMovies','Contract','PaperlessBilling','PaymentMethod','Churn']
churn_df[l] = churn_df[l].apply(pd.to_numeric)

In [51]:
churn_df_X = churn_df.copy()
churn_df_y =  churn_df_X.pop('Churn')

In [52]:
churn_df_X.pop("customerID")

0       7590-VHVEG
1       5575-GNVDE
2       3668-QPYBK
3       7795-CFOCW
4       9237-HQITU
           ...    
7038    6840-RESVB
7039    2234-XADUH
7040    4801-JZAZL
7041    8361-LTMKD
7042    3186-AJIEK
Name: customerID, Length: 7043, dtype: object

In [53]:
churn_df_y=pd.to_numeric(churn_df_y).astype(int)
X_train, X_test, y_train, y_test = train_test_split(churn_df_X, churn_df_y, train_size=0.80)

In [54]:
import autosklearn.classification

In [55]:
automl = autosklearn.classification.AutoSklearnClassifier(time_left_for_this_task=1200, per_run_time_limit=30, n_jobs=2,
    include = {'classifier' : ["random_forest", "sgd"]})


In [56]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5634 entries, 2949 to 578
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   gender            5634 non-null   int64  
 1   SeniorCitizen     5634 non-null   int64  
 2   Partner           5634 non-null   int64  
 3   Dependents        5634 non-null   int64  
 4   tenure            5634 non-null   int64  
 5   PhoneService      5634 non-null   int64  
 6   MultipleLines     5634 non-null   int64  
 7   InternetService   5634 non-null   int64  
 8   OnlineSecurity    5634 non-null   int64  
 9   OnlineBackup      5634 non-null   int64  
 10  DeviceProtection  5634 non-null   int64  
 11  TechSupport       5634 non-null   int64  
 12  StreamingTV       5634 non-null   int64  
 13  StreamingMovies   5634 non-null   int64  
 14  Contract          5634 non-null   int64  
 15  PaperlessBilling  5634 non-null   int64  
 16  PaymentMethod     5634 non-null   int64 

In [57]:
automl.fit(X_train, y_train, X_test, y_test)

AutoSklearnClassifier(include={'classifier': ['random_forest', 'sgd']},
                      n_jobs=2, per_run_time_limit=30,
                      time_left_for_this_task=1200)

In [58]:
print(automl.show_models())

{102: {'model_id': 102, 'rank': 1, 'cost': 0.1919354838709677, 'ensemble_weight': 0.08, 'data_preprocessor': <autosklearn.pipeline.components.data_preprocessing.DataPreprocessorChoice object at 0x7ffa1a78a890>, 'balancing': Balancing(random_state=1), 'feature_preprocessor': <autosklearn.pipeline.components.feature_preprocessing.FeaturePreprocessorChoice object at 0x7ffa1b878d50>, 'classifier': <autosklearn.pipeline.components.classification.ClassifierChoice object at 0x7ffa1b88bfd0>, 'sklearn_classifier': RandomForestClassifier(criterion='entropy', max_features=6, min_samples_leaf=15,
                       min_samples_split=4, n_estimators=512, n_jobs=1,
                       random_state=1, warm_start=True)}, 104: {'model_id': 104, 'rank': 2, 'cost': 0.206989247311828, 'ensemble_weight': 0.02, 'data_preprocessor': <autosklearn.pipeline.components.data_preprocessing.DataPreprocessorChoice object at 0x7ffa1ae55b10>, 'balancing': Balancing(random_state=1), 'feature_preprocessor': <auto

In [59]:
print(automl.sprint_statistics())

auto-sklearn results:
  Dataset name: cdd8b8da-0887-11ed-83f6-0242ac1c0002
  Metric: accuracy
  Best validation score: 0.808065
  Number of target algorithm runs: 125
  Number of successful target algorithm runs: 106
  Number of crashed target algorithm runs: 2
  Number of target algorithms that exceeded the time limit: 17
  Number of target algorithms that exceeded the memory limit: 0



In [60]:
automl.cv_results_['params'][np.argmax(automl.cv_results_['mean_test_score'])]

{'balancing:strategy': 'none',
 'classifier:__choice__': 'random_forest',
 'classifier:random_forest:bootstrap': 'True',
 'classifier:random_forest:criterion': 'entropy',
 'classifier:random_forest:max_depth': 'None',
 'classifier:random_forest:max_features': 0.3438929807547332,
 'classifier:random_forest:max_leaf_nodes': 'None',
 'classifier:random_forest:min_impurity_decrease': 0.0,
 'classifier:random_forest:min_samples_leaf': 15,
 'classifier:random_forest:min_samples_split': 4,
 'classifier:random_forest:min_weight_fraction_leaf': 0.0,
 'data_preprocessor:__choice__': 'feature_type',
 'data_preprocessor:feature_type:categorical_transformer:categorical_encoding:__choice__': 'one_hot_encoding',
 'data_preprocessor:feature_type:categorical_transformer:category_coalescence:__choice__': 'no_coalescense',
 'data_preprocessor:feature_type:numerical_transformer:imputation:strategy': 'most_frequent',
 'data_preprocessor:feature_type:numerical_transformer:rescaling:__choice__': 'robust_scal

In [61]:
predictions = automl.predict(X_test)

In [62]:
import sklearn.metrics
print("Accuracy score", sklearn.metrics.accuracy_score(y_test, predictions))

Accuracy score 0.8048261178140526
